# 🚀 Hướng dẫn Deploy dự án Document Layout Analysis
**Tác giả:** Nam Pham Nhat
**Mục tiêu:** Chuyển đổi Model YOLOv8 từ Local lên Streamlit Cloud.

---

## 1. Thiết lập Môi trường (Environment)
Do dự án sử dụng các thư viện AI nặng (Torch, Ultralytics), cần tránh xung đột với Python hệ thống (đặc biệt là bản 3.13/3.14 chưa ổn định).

### Quy trình chuẩn:
1.  **Cài đặt Python 3.11:** Tải từ trang chủ python.org.
2.  **Tạo môi trường ảo (Virtual Environment):**
    ```bash
    py -3.11 -m venv venv
    ```
3.  **Cấp quyền chạy Script (Windows):**
    ```powershell
    Set-ExecutionPolicy -ExecutionPolicy RemoteSigned -Scope Process
    ```
4.  **Kích hoạt môi trường:**
    ```bash
    .\venv\Scripts\activate
    ```
5.  **Cài đặt thư viện:**
    ```bash
    pip install streamlit ultralytics pillow opencv-python-headless
    ```

In [ ]:
%%writefile app.py
import streamlit as st
from ultralytics import YOLO
from PIL import Image
import io
from collections import Counter
import numpy as np

# --- 1. Cấu hình trang ---
st.set_page_config(page_title="DocLayNet Analysis", page_icon="📄", layout="wide")

# --- 2. Cấu hình Ngôn ngữ ---
LANGUAGES = {
    "English": {
        "title": "📄 Document Layout Detection App",
        "btn_analyze": "🚀 Analyze Layout",
        "processing": "Analyzing...",
        "col_orig": "Original Image",
        "col_res": "Detected Result",
        "download_btn": "⬇️ Download Result",
        "stats_header": "📊 Statistics",
        "error_load": "Error loading model!",
        "no_obj": "No objects detected!",
    },
    "Tiếng Việt": {
        "title": "📄 Phân tích Bố cục Tài liệu",
        "btn_analyze": "🚀 Phân tích ngay",
        "processing": "Đang xử lý...",
        "col_orig": "Ảnh gốc",
        "col_res": "Kết quả",
        "download_btn": "⬇️ Tải ảnh về",
        "stats_header": "📊 Thống kê",
        "error_load": "Lỗi tải model!",
        "no_obj": "Không tìm thấy đối tượng nào!",
    }
}

# Sidebar
with st.sidebar:
    st.header("Cài đặt / Settings")
    lang_code = st.selectbox("Ngôn ngữ / Language", ["English", "Tiếng Việt"], index=0)
    text = LANGUAGES[lang_code]

st.title(text["title"])

# --- 3. Load Model (Có cache) ---
@st.cache_resource
def load_model():
    try:
        # Load model YOLO
        model = YOLO("best.pt")
        return model
    except Exception as e:
        return None

model = load_model()

if model is None:
    st.error(f"❌ {text['error_load']} (Không tìm thấy file 'best.pt')")
    st.stop()

# --- 4. Upload & Xử lý ---
uploaded_file = st.file_uploader("Upload Image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    try:
        # Convert sang RGB để tránh lỗi kênh màu
        image = Image.open(uploaded_file).convert("RGB")
        
        col1, col2 = st.columns(2)
        with col1:
            st.subheader(text["col_orig"])
            st.image(image, use_container_width=True)

        st.write("")
        if st.button(text["btn_analyze"], type="primary", use_container_width=True):
            
            with st.spinner(text["processing"]):
                # Chạy Model
                results = model(image, conf=0.25)
                
                if len(results[0].boxes) == 0:
                    st.warning(text["no_obj"])
                else:
                    # Plot ảnh kết quả
                    res_plotted = results[0].plot()[:, :, ::-1]
                    res_image = Image.fromarray(res_plotted)

                    with col2:
                        st.subheader(text["col_res"])
                        st.image(res_image, use_container_width=True)

                        # Nút download
                        buf = io.BytesIO()
                        res_image.save(buf, format="PNG")
                        byte_im = buf.getvalue()
                        
                        st.download_button(
                            label=text["download_btn"],
                            data=byte_im,
                            file_name="result.png",
                            mime="image/png",
                            use_container_width=True
                        )

                    # Thống kê
                    st.divider()
                    st.subheader(text["stats_header"])
                    class_names = [model.names[int(box.cls[0])] for box in results[0].boxes]
                    counts = Counter(class_names)
                    
                    cols = st.columns(4)
                    for i, (name, count) in enumerate(counts.items()):
                        with cols[i % 4]:
                            st.metric(label=name, value=count)

    except Exception as e:
        st.error(f"❌ Error: {e}")

Link web: https://document-image-detection.streamlit.app/

In [ ]:
%%writefile requirements.txt
streamlit
ultralytics
pillow
numpy
opencv-python-headless

In [ ]:
%%writefile packages.txt
libgl1-mesa-glx
libglib2.0-0

## 2. Các bước Deploy lên Streamlit Cloud
Sau khi đã có đủ các file (`app.py`, `requirements.txt`, `packages.txt`, `best.pt`), thực hiện:

1.  **GitHub:**
    * Tạo Repository mới (Public).
    * Upload toàn bộ các file trên vào repo.
    
2.  **Streamlit Cloud:**
    * Truy cập [share.streamlit.io](https://share.streamlit.io).
    * New App -> Chọn Repo GitHub vừa tạo.
    * Nhấn **Deploy**.

### ⚠️ Lưu ý quan trọng:
* Nếu gặp lỗi `libGL...`: Kiểm tra xem đã có file `packages.txt` chưa.
* Nếu gặp lỗi Python version: Vào Settings trên Streamlit Cloud -> Chỉnh Python thành **3.11**.